In [ ]:
import intake
from easygems import healpix as egh

import numpy as np

import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cf

import warnings

warnings.filterwarnings("ignore", category=FutureWarning)

In [ ]:
# Time period

time = ('2020-08-01','2020-08-31')

# Region

domains10x10 = {
    "peruvian":     np.array([-90, -80, -20, -10]) ,
    "namibian":     np.array([0, 10, -20, -10]),
    "californian":  np.array([-130, -120, 20, 30]),
    "canarian":     np.array([-35, -25, 15, 25])
}

map_domain = domains10x10['namibian'] + np.array([-1,1,-1,1])*10

In [ ]:
# Load dataset

hknode = 'EU'
sim = 'icon_d3hp003'
zoom = 8


cat = intake.open_catalog("https://digital-earths-global-hackathon.github.io/catalog/catalog.yaml")[hknode]

ds = cat[sim](time_method='inst',time='PT6H',zoom=zoom).to_dask().pipe(egh.attach_coords)

In [ ]:
# Select time period and and region

cells = egh.isel_extent(ds,map_domain+np.array([-1,1,-1,1])) # + 1 deg margin on each side

ds = ds.sel(time=slice(*time)).isel(cell=cells)

In [ ]:
%%time

# Integrate a 3D variable in vertical

ds['qallvi'] = (ds['qall'].integrate('pressure')/9.81) \
    .assign_attrs(long_name='Total condensate', units='kg/m^2')

In [ ]:
# Plot maps

var = 'qallvi'   # variable
cmap = 'Blues_r' # colormap
vmax = 1         # max level on the color scale

time_step = np.timedelta64(10,'D') # Plot a map every 10 days


plot_path = f"./figures/maps/zoom{zoom:d}/"
figsize = (8,6)


for t in np.arange(ds.time[0].values,ds.time[-1].values,time_step):
    
    fig, ax = plt.subplots(figsize=figsize,
                            subplot_kw={"projection": ccrs.PlateCarree()},
                            constrained_layout=True)

    ax.set_extent(map_domain, crs=ccrs.PlateCarree())

    im = egh.healpix_show(ds[var].sel(time=t), ax=ax, cmap=cmap, vmin=0, vmax=vmax)
    
    ax.add_feature(cf.NaturalEarthFeature('physical', 'land', '10m'),
                   facecolor='none', edgecolor='red', linewidth=1)
    gl = ax.gridlines(draw_labels=True)
    gl.top_labels = False
    gl.right_labels = False

    cb = plt.colorbar(im, ax=ax, shrink=0.9, aspect=30, pad=0.02, extend='max',
                      label=f"{ds[var].attrs['long_name']:s} [{ds[var].attrs['units']:s}]")

    datestr = t.astype('datetime64[h]').item().strftime('%Y-%m-%dT%H')
    ax.set_title(datestr+f"  zoom={zoom:d}")
    
    plt.savefig(plot_path+var+'_'+datestr,bbox_inches='tight',dpi=300)
    
    print(datestr)
    # plt.close(fig) # Uncomment if you don't want to see the plots in jupyter but only save to files